<a href="https://colab.research.google.com/github/nmitra28/Audio_LLM/blob/main/audial_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import json
import torch
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AutoFeatureExtractor, Wav2Vec2ConformerForXVector, AutoModelForCausalLM, AutoTokenizer

**Step 1 :** Loading dataset LibriSpeech. It is a corpus of approximately 1000 hours of 16kHz read English speech, prepared by Vassil Panayotov with the assistance of Daniel Povey. The data is derived from read audiobooks from the LibriVox project, and has been carefully segmented and aligned.

**Data Fields**

**file:** A path to the downloaded audio file in .flac format.

**audio:** A dictionary containing the path to the downloaded audio file, the decoded audio array, and the sampling rate. Note that when accessing the audio column: dataset[0]["audio"] the audio file is automatically decoded and resampled to dataset.features["audio"].sampling_rate. Decoding and resampling of a large number of audio files might take a significant amount of time. Thus it is important to first query the sample index before the "audio" column, i.e. dataset[0]["audio"] should always be preferred over dataset["audio"][0].

**text:** the transcription of the audio file.

**id:** unique id of the data sample.

**speaker_id:** unique id of the speaker. The same speaker id can be found for multiple data samples.

chapter_id: id of the audiobook chapter which includes the transcription.




In [ ]:
from datasets import load_dataset

# Available splits: ['train.clean.100', 'train.clean.360', 'train.other.500', 'validation.clean', 'validation.other', 'test.clean', 'test.other']
librispeech = load_dataset("librispeech_asr",  split="train.clean.100", streaming=True)
print(next(iter(librispeech)))

{'file': '374-180298-0000.flac', 'audio': {'path': '374-180298-0000.flac', 'array': array([ 7.01904297e-04,  7.32421875e-04,  7.32421875e-04, ...,
       -2.74658203e-04, -1.83105469e-04, -3.05175781e-05]), 'sampling_rate': 16000}, 'text': 'CHAPTER SIXTEEN I MIGHT HAVE TOLD YOU OF THE BEGINNING OF THIS LIAISON IN A FEW LINES BUT I WANTED YOU TO SEE EVERY STEP BY WHICH WE CAME I TO AGREE TO WHATEVER MARGUERITE WISHED', 'speaker_id': 374, 'chapter_id': 180298, 'id': '374-180298-0000'}


[512,1]

In [ ]:
iter = iter(librispeech)

**facebook/wav2vec2-conformer-rope-large-960h-ft** : A ***finetuned Wav2Vec2 Conformer*** with rotary position embeddings, pretrained and fine-tuned on 960 hours of Librispeech on 16kHz sampled speech audio. When using the model make sure that your speech input is also sampled at 16Khz.

In [ ]:
dataset = librispeech

#model for audio
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-conformer-rope-large-960h-ft")
model_audio = Wav2Vec2ConformerForXVector.from_pretrained("facebook/wav2vec2-conformer-rope-large-960h-ft")

#model for texts
llm = AutoModelForCausalLM.from_pretrained("huggyllama/llama-7b")
tokenizer = AutoTokenizer.from_pretrained("huggyllama/llama-7b")

preprocessor_config.json:   0%|          | 0.00/254 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.24k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.37G [00:00<?, ?B/s]

Some weights of Wav2Vec2ConformerForXVector were not initialized from the model checkpoint at facebook/wav2vec2-conformer-rope-large-960h-ft and are newly initialized: ['tdnn.1.kernel.bias', 'classifier.weight', 'tdnn.3.kernel.bias', 'wav2vec2_conformer.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'tdnn.2.kernel.bias', 'projector.weight', 'tdnn.0.kernel.weight', 'tdnn.4.kernel.bias', 'tdnn.4.kernel.weight', 'tdnn.1.kernel.weight', 'projector.bias', 'feature_extractor.weight', 'tdnn.0.kernel.bias', 'tdnn.3.kernel.weight', 'feature_extractor.bias', 'classifier.bias', 'objective.weight', 'tdnn.2.kernel.weight', 'wav2vec2_conformer.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [ ]:
#Iterator for dataset
SAMPLING_RATE = dataset.features["audio"].sampling_rate
data=[]
# audio file is decoded on the fly
for d in range(100):
    a = next(iter)

    #Get audial embeddings here
    inputs = feature_extractor(
        [a["audio"]["array"]], sampling_rate=SAMPLING_RATE, return_tensors="pt", padding=True
    )
    with torch.no_grad():
        speech_embeddings = model_audio(**inputs).embeddings
    speech_embeddings = torch.nn.functional.normalize(speech_embeddings, dim=-1).cpu()

    #Get Token and Textual Embeddings here
    text_input_id = tokenizer.encode(a["text"], return_tensors="pt")
    text_embeddings = llm.model.embed_tokens(text_input_id)

    #pad speech embeddings and concat embeddings
    speech_embeddings_pad = torch.nn.functional.pad(speech_embeddings[None,:, :], (0,text_embeddings.shape[2]-512,0,text_embeddings.shape[1]-1), "constant", 0)
    print(speech_embeddings_pad.shape)
    print(text_embeddings.shape)
    both_embeddings = torch.cat((speech_embeddings_pad, text_embeddings),dim = 0)

    #generate output
    generated_output = llm.generate(text_input_id.repeat(2, 1), inputs_embeds=both_embeddings)
    output_decoded = tokenizer.decode(generated_output[0])
                                                    # parameter ^  skip_special_tokens=True

    #adding to dataframe
    temp = speech_embeddings.tolist()
    new_row = {'both_embeddings': both_embeddings, 'speech_embeddings': temp[0], 'text_embeddings': text_embeddings.tolist(), 'generated_output_tensor' : generated_output, 'decoded_output' : output_decoded, 'id': a["id"], 'file' :  a["file"], 'input_text':a["text"]}
    data.append(new_row)

torch.Size([1, 78, 4096])
torch.Size([1, 78, 4096])


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 78, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


torch.Size([1, 86, 4096])
torch.Size([1, 86, 4096])


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 86, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


torch.Size([1, 74, 4096])
torch.Size([1, 74, 4096])


...

torch.Size([1, 72, 4096])
torch.Size([1, 72, 4096])
torch.Size([1, 84, 4096])
torch.Size([1, 84, 4096])
torch.Size([1, 108, 4096])
torch.Size([1, 108, 4096])


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 108, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


torch.Size([1, 102, 4096])
torch.Size([1, 102, 4096])


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 102, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


torch.Size([1, 99, 4096])
torch.Size([1, 99, 4096])
torch.Size([1, 61, 4096])
torch.Size([1, 61, 4096])
torch.Size([1, 73, 4096])
torch.Size([1, 73, 4096])
torch.Size([1, 83, 4096])
torch.Size([1, 83, 4096])
torch.Size([1, 97, 4096])
torch.Size([1, 97, 4096])
torch.Size([1, 85, 4096])
torch.Size([1, 85, 4096])
torch.Size([1, 74, 4096])
torch.Size([1, 74, 4096])
torch.Size([1, 81, 4096])
torch.Size([1, 81, 4096])
torch.Size([1, 90, 4096])
torch.Size([1, 90, 4096])
torch.Size([1, 85, 4096])
torch.Size([1, 85, 4096])
torch.Size([1, 86, 4096])
torch.Size([1, 86, 4096])
torch.Size([1, 85, 4096])
torch.Size([1, 85, 4096])


In [ ]:
new_row["input_text"]

'SAID SHE TURNING TO PRUDENCE AND SPEAKING IN A BROKEN VOICE YOU CAN REPORT THIS SCENE TO THE DUKE AND YOU CAN ADD THAT WE HAVE NO LONGER NEED OF HIM FROM THAT DAY FORTH THE DUKE WAS NEVER REFERRED TO'

In [ ]:
new_row["decoded_output"]

'<s> SAID SHE TURNING TO PRUDENCE AND SPEAKING IN A BROKEN VOICE YOU CAN REPORT THIS SCENE TO THE DUKE AND YOU CAN ADD THAT WE HAVE NO LONGER NEED OF HIM FROM THAT DAY FORTH THE DUKE WAS NEVER REFERRED TO1'

In [ ]:
df = pd.json_normalize(data)

In [ ]:
# df.to_csv('100.csv', index=False)

In [ ]:
df[:10]

,both_embeddings,speech_embeddings,text_embeddings,generated_output_tensor,decoded_output,id,file,input_text
0,"[[[tensor(0.0686, grad_fn=<UnbindBackward0>), ...","[0.0686289593577385, 0.02398112788796425, -0.0...","[[[0.015289306640625, -0.01215362548828125, 0....","[[tensor(1), tensor(12689), tensor(317), tenso...",<s> CHAPTER SIXTEEN I MIGHT HAVE TOLD YOU OF T...,374-180298-0000,374-180298-0000.flac,CHAPTER SIXTEEN I MIGHT HAVE TOLD YOU OF THE B...
1,"[[[tensor(0.0629, grad_fn=<UnbindBackward0>), ...","[0.06286539137363434, 0.01988127827644348, -0....","[[[0.015289306640625, -0.01215362548828125, 0....","[[tensor(1), tensor(23851), tensor(29954), ten...",<s> MARGUERITE TO BE UNABLE TO LIVE APART FROM...,374-180298-0001,374-180298-0001.flac,MARGUERITE TO BE UNABLE TO LIVE APART FROM ME ...
2,"[[[tensor(0.0681, grad_fn=<UnbindBackward0>), ...","[0.06807733327150345, 0.01830965094268322, -0....","[[[0.015289306640625, -0.01215362548828125, 0....","[[tensor(1), tensor(306), tensor(399), tensor(...",<s> I WISHED ABOVE ALL NOT TO LEAVE MYSELF TIM...,374-180298-0002,374-180298-0002.flac,I WISHED ABOVE ALL NOT TO LEAVE MYSELF TIME TO...
3,"[[[tensor(0.0708, grad_fn=<UnbindBackward0>), ...","[0.07082702219486237, 0.025349341332912445, -0...","[[[0.015289306640625, -0.01215362548828125, 0....","[[tensor(1), tensor(319), tensor(1799), tensor...",<s> ASSUMED ALL AT ONCE AN APPEARANCE OF NOISE...,374-180298-0003,374-180298-0003.flac,ASSUMED ALL AT ONCE AN APPEARANCE OF NOISE AND...
4,"[[[tensor(0.0741, grad_fn=<UnbindBackward0>), ...","[0.0741242989897728, 0.026576295495033264, -0....","[[[0.015289306640625, -0.01215362548828125, 0....","[[tensor(1), tensor(6058), tensor(29950), tens...",<s> NOTHING IS SO EXPENSIVE AS THEIR CAPRICES ...,374-180298-0004,374-180298-0004.flac,NOTHING IS SO EXPENSIVE AS THEIR CAPRICES FLOW...
5,"[[[tensor(0.0735, grad_fn=<UnbindBackward0>), ...","[0.07352039963006973, 0.022867891937494278, -0...","[[[0.015289306640625, -0.01215362548828125, 0....","[[tensor(1), tensor(19519), tensor(383), tenso...",<s> MY FATHER WAS AND STILL IS RECEVEUR GENERA...,374-180298-0005,374-180298-0005.flac,MY FATHER WAS AND STILL IS RECEVEUR GENERAL AT...
6,"[[[tensor(0.0703, grad_fn=<UnbindBackward0>), ...","[0.07028435170650482, 0.02042168378829956, -0....","[[[0.015289306640625, -0.01215362548828125, 0....","[[tensor(1), tensor(306), tensor(12766), tenso...",<s> I CAME TO PARIS STUDIED LAW WAS CALLED TO ...,374-180298-0006,374-180298-0006.flac,I CAME TO PARIS STUDIED LAW WAS CALLED TO THE ...
7,"[[[tensor(0.0685, grad_fn=<UnbindBackward0>), ...","[0.06851773709058762, 0.023884762078523636, -0...","[[[0.015289306640625, -0.01215362548828125, 0....","[[tensor(1), tensor(19519), tensor(8528), tens...",<s> MY EXPENSES WERE VERY MODERATE ONLY I USED...,374-180298-0007,374-180298-0007.flac,MY EXPENSES WERE VERY MODERATE ONLY I USED UP ...
8,"[[[tensor(0.0697, grad_fn=<UnbindBackward0>), ...","[0.06968538463115692, 0.024949349462985992, -0...","[[[0.015289306640625, -0.01215362548828125, 0....","[[tensor(1), tensor(15842), tensor(6093), tens...",<s> FOR THE REST NOT A PENNY OF DEBT THIS THEN...,374-180298-0008,374-180298-0008.flac,FOR THE REST NOT A PENNY OF DEBT THIS THEN WAS...
9,"[[[tensor(0.0724, grad_fn=<UnbindBackward0>), ...","[0.07243561744689941, 0.02593766152858734, -0....","[[[0.015289306640625, -0.01215362548828125, 0....","[[tensor(1), tensor(23851), tensor(29954), ten...",<s> MARGUERITE'S NATURE WAS VERY CAPRICIOUS AN...,374-180298-0009,374-180298-0009.flac,MARGUERITE'S NATURE WAS VERY CAPRICIOUS AND LI...


In [ ]:
notes from Dave:

1. Write a func audio embedding (512d),

SyntaxError: invalid syntax (<ipython-input-1-b7c6bfd2c7f3>, line 1)

In [ ]:
df1 = df1.drop( columns=['both_embeddings',	'speech_embeddings',	'text_embeddings',	'generated_output_tensor'])

In [ ]:
df1.to_csv('10_outputs.csv', index=False)

In [ ]:
#Iterator for dataset
SAMPLING_RATE = dataset.features["audio"].sampling_rate
data=[]
# audio file is decoded on the fly
for d in range(1):
    a = next(iter)

    #Get audial embeddings here
    inputs = feature_extractor(
        [a["audio"]["array"]], sampling_rate=SAMPLING_RATE, return_tensors="pt", padding=True
    )
    with torch.no_grad():
        speech_embeddings = model_audio(**inputs).embeddings
    speech_embeddings = torch.nn.functional.normalize(speech_embeddings, dim=-1).cpu()

    #Get Token and Textual Embeddings here
    text_input_id = tokenizer.encode(a["text"], return_tensors="pt")
    text_embeddings = llm.model.embed_tokens(text_input_id)

    #pad speech embeddings and concat embeddings
    speech_embeddings_pad = torch.nn.functional.pad(speech_embeddings[None,:, :], (0,text_embeddings.shape[2]-512,0,text_embeddings.shape[1]-1), "constant", 0)
    print(speech_embeddings_pad.shape)
    print(text_embeddings.shape)
    both_embeddings = torch.cat((speech_embeddings_pad, text_embeddings),dim = 0)

    #generate output
    generated_output = llm.generate( inputs_embeds=text_embeddings)
    output_decoded = tokenizer.decode(generated_output[0])
                                                    # parameter ^  skip_special_tokens=True

    #adding to dataframe
    temp = speech_embeddings.tolist()
    new_row = {'both_embeddings': both_embeddings, 'speech_embeddings': temp[0], 'text_embeddings': text_embeddings.tolist(), 'generated_output_tensor' : generated_output, 'decoded_output' : output_decoded, 'id': a["id"], 'file' :  a["file"], 'input_text':a["text"]}
    data.append(new_row)

torch.Size([1, 76, 4096])
torch.Size([1, 76, 4096])


In [ ]:
new_row["decoded_output"]

'<s>.\nMY FATHER IS A MAN OF GREAT INTEGRITY AND'

In [ ]:
new_row["input_text"]

'MY FATHER WAS AND STILL IS RECEVEUR GENERAL AT C HE HAS A GREAT REPUTATION THERE FOR LOYALTY THANKS TO WHICH HE WAS ABLE TO FIND THE SECURITY WHICH HE NEEDED IN ORDER TO ATTAIN THIS POSITION'